In [96]:
import numpy as np
import pandas as pd
import sklearn
import nltk

In [97]:
data = pd.read_csv("combined_data.csv")
data

,sentiment,Text
0,sadness,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,happy,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
61454,fear,Melissa stared at her friend in dism
61455,happy,Successive state elections have seen the gover...
61456,fear,Vincent was irritated but not dismay
61457,happy,Kendall-Hume turned back to face the dismayed ...


In [98]:
data["sentiment"].value_counts()

fear        16241
happy       13508
sadness      9796
neutral      8960
love         4720
anger        4069
surprise     2639
relief       1526
Name: sentiment, dtype: int64

In [99]:
# Removing punctuation, URL, and tags
import re
data['Text'] = data['Text'].apply(lambda x: re.sub("(@[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)",'',x))


In [100]:
data.dropna(inplace=True)

In [101]:
print(data.shape)
data.head()
print(data.tail())

(61459, 2)
      sentiment                                               Text
61454      fear               Melissa stared at her friend in dism
61455     happy  Successive state elections have seen the gover...
61456      fear               Vincent was irritated but not dismay
61457     happy  KendallHume turned back to face the dismayed coup
61458     happy                     I am dismayed  but not surpris


In [102]:
import snowballstemmer
ss = snowballstemmer.stemmer('english')
def replace(x):
    words = x.split()
    newtext = ''
    for w in words:
        n = ss.stemWord(w)
        newtext += n
        newtext += " "
    return newtext
data['Text'] = data['Text'].apply(lambda x: replace(x))
data.head()

,sentiment,Text
0,sadness,i know i was listenin to bad habit earlier and...
1,sadness,Layin n bed with a headach ughhhhwaitin on you...
2,sadness,Funer ceremonygloomi friday
3,happy,want to hang out with friend SOON
4,neutral,We want to trade with someon who has Houston t...


In [103]:

data['sentiment'].value_counts()



fear        16241
happy       13508
sadness      9796
neutral      8960
love         4720
anger        4069
surprise     2639
relief       1526
Name: sentiment, dtype: int64

In [104]:
#build new non-skewed dataset

#sample with replacement
def stratify(data, N):
    rows = []
    fear = data[data['sentiment'] == 'fear']
    happy = data[data['sentiment'] == 'happy']
    sad = data[data['sentiment'] == 'sadness']
    neutral = data[data['sentiment'] == 'neutral']
    love = data[data['sentiment'] == 'love']
    anger = data[data['sentiment'] == 'anger']
    surprise = data[data['sentiment'] == 'surprise']
    relief = data[data['sentiment'] == 'relief']
    
    for i in range(N):
        #print(fear.loc[np.random.choice(fear.index)])
        rows.append(fear.loc[np.random.choice(fear.index)])
        rows.append(happy.loc[np.random.choice(happy.index)])
        rows.append(sad.loc[np.random.choice(sad.index)])
        rows.append(neutral.loc[np.random.choice(neutral.index)])
        rows.append(love.loc[np.random.choice(love.index)])
        rows.append(anger.loc[np.random.choice(anger.index)])
        rows.append(surprise.loc[np.random.choice(surprise.index)])
        rows.append(relief.loc[np.random.choice(relief.index)])
    sentiments = [x['sentiment'] for x in rows]
    texts = [x['Text'] for x in rows]
    d = {'sentiment': sentiments, 'Text': texts}
    return pd.DataFrame(d)
    



In [105]:
data = stratify(data, 10000)
data['sentiment'].value_counts()

neutral     10000
fear        10000
happy       10000
surprise    10000
anger       10000
love        10000
relief      10000
sadness     10000
Name: sentiment, dtype: int64

#### Split Train and Test sets:

In [106]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], 
                                                    data['sentiment'],test_size=0.10, 
                                                    random_state=1)


#### Word of Bags: Preprocessing

In [107]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Instantiate the CountVectorizer method


tfid_vector = TfidfVectorizer(stop_words = 'english')
count_vector = CountVectorizer(stop_words='english', max_features=10000)


# Fit the training data and then return the matrix
training_data = tfid_vector.fit_transform(X_train)

# Transform testing data and return the matrix. 
testing_data = tfid_vector.transform(X_test)

In [108]:
print(training_data.shape)


(72000, 25631)


### Multinomial Naive Bayes

Turn the Dataset into a list of P(sentiment | word) = P(word | sentiment) * p(sentiment) / p(word)

In [109]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data,y_train)

MultinomialNB()

In [110]:
predictions = naive_bayes.predict(testing_data)

In [111]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(y_test, predictions)))
print('precision score: {}'.format(precision_score(y_test, predictions, average='macro')))
print('recall score: {}'.format(recall_score(y_test, predictions,average='macro')))
print('f1 score: {}'.format(f1_score(y_test, predictions,average='macro')))



Accuracy score: 0.561
precision score: 0.5530033917768541
recall score: 0.5620283967231994
f1 score: 0.5491992204206585


In [64]:
counts = data['sentiment'].value_counts()

In [65]:
#count_vector_ns = CountVectorizer(stop_words='english', max_features=5000)
vectData = tfid_vector.transform(X_train)

            
    

In [66]:
rowsum = vectData.sum(axis=1)
colsum = vectData.sum(axis=0)
wordsum = vectData.sum()

In [79]:
print(vectData.shape[0])
print(X_train.at[10])


57600


KeyError: 10

In [76]:
sentiment_sums = []
for sentiment in counts.index:
    sentsum = None
    for i in range(vectData.shape[0]):
        row = vectData.getrow(i)
        print(y_train.at[i])
        if y_train.at[i] == sentiment:
            if sentsum == None:
                sentsum = row
            else:
                sentsum += row
    sentiment_sums.append(sentsum.todense())

        

fear
happy
sadness
neutral
love
anger
surprise
relief
fear
happy


KeyError: 10

In [ ]:
import random
feature_names = count_vector_ns.get_feature_names()
sentiment_col_sums = []
for s in sentiment_sums:
    sentiment_col_sums.append(s.sum(axis=1))


def getWordIndex(w):
    try:
        idx = feature_names.index(w)
        return idx
    except ValueError:
        return -1
    
def predict(text):
    stemmer = PorterStemmer()
    running_probs = [1 for i in range(len(counts.index))]
    words = text.split()
    for w in words:
        w = stemmer.stem(w)
        idx = getWordIndex(w)
        pw = colsum.A[0][idx]/wordsum
        for i in range(len(counts.index)):
            if idx == -1:
                #print('word not found:', w)
                break # dont influence probs if word unknown
            ps = counts[i]
            pwgs = sentiment_sums[i].A[0][idx]/sentiment_col_sums[i]
            prob = (pwgs*ps)/pw
            if prob == 0:
                prob = 10**-7
            running_probs[i] *= prob
    maxprob = max(running_probs)
    maxes = []
    for idx,p in enumerate(running_probs):
        if p == maxprob:
            maxes.append(idx)
            
    return counts.index[random.choice(maxes)]

def test(testdata, labels):
    correct = 0
    for i,text in enumerate(testdata):
        label = predict(text)
        #print(f'pred:{label}, act:{labels[labels.index[i]]}')
        if label == labels[labels.index[i]]:
            correct += 1
    return correct/len(labels)
        
    

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['Text'], 
                                                    data['sentiment'],test_size=0.10, 
                                                    random_state=1)

test(X_test,y_test)

In [ ]:
predict('wow i didnt expect that')